<a href="https://colab.research.google.com/github/BrendanL72/Hospitalization-Risk/blob/main/arimamodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 3.2 MB/s 
     |████████████████████████████████| 9.9 MB 30.4 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [3]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [17]:
dfadl = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/MobileEdgeTaskCodeVisitSummaryJanuary-AugustSortedAfterDeIdentify.xlsx')
dfadl.nunique()

DeIdentify ID               210
ProviderID                    3
PayerGroup                    6
Caregiver ID                191
VisitDuration               307
ServiceTypeName              15
VisitDate                   243
ActualTimeIn                412
ActualTimeOut               390
txtBreakTimesValue            0
VisitDate2                  243
ScheduledTimeIn             148
ScheduledTimeOut            148
TaskCodeGroupName            60
HasBeenDone                   2
Reason                        7
Comments                   2490
NoSignature                   2
RectangleVoiceCaptured        1
SignatureDate             22726
dtype: int64

In [18]:
dfcrf = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/ClientReviewFormRemovingData.xlsx')
dfcrf.nunique()

Time                                               369
Masked Client ID                                   167
Date of Review                                     144
COVID -19                                            2
Which Review is this                                 3
                                                  ... 
Details of Schedule changes                         20
Are you happy with the services being provided?      2
If No make notes here                                1
Any Questions or Concerns?                          98
Case Manager Notes                                 243
Length: 67, dtype: int64

In [19]:
# clientarr = dfcrf['Masked Client ID'].unique()
# devicearr = df['device_id'].unique()
# #assign empty client column to sensor dataframe
# df['client'] = ''
# #fill df['client'] with random client ids corresponding to device id
# """for i in range(0, len(devicearr)):
#   for j in range(0, len(df)):
#     if(df['device_id'][j] == devicearr[i]):
#       df['client'][j] = dfcrf['Masked Client ID'][i]"""
# it = 0
# step = int(len(df)/dfcrf['Masked Client ID'].nunique())
# for i in range(0, len(df), step):
#   for j in range(0,step):
#     df['client'][i+j] = clientarr[it]
    
#   it = it + 1

In [20]:
dfcrf.head()


,Time,Masked Client ID,Date of Review,COVID -19,Which Review is this,Insurance,Any Hospitalizations in the last 30 days,Note on Hospitalization (date and reason),How can we help prevent a return to hospital?,Have you fallen in:,...,Does your Caregiver carry out the care plan satisfactory,Detail any issues,Are the cargivers able to perform all the tasks on your care plan,Detail any issues.1,Are there any upcoming schedule changes?,Details of Schedule changes,Are you happy with the services being provided?,If No make notes here,Any Questions or Concerns?,Case Manager Notes
0,2022-01-03 14:59:17,Client088,2022-01-03,By Phone,5 Day,Banner,No,NaN,NaN,No falls,...,Yes,NaN,Yes,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,"ALL IS WELL FCG, SEEKING SUPPORT FOR MED APPTS..."
1,2022-01-04 13:12:24,Client172,2022-01-04,By Phone,90 Day,UHC,No,NaN,NaN,No falls,...,NaN,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,"CHECKING INTO DD SERVICES, HE HAS APPLIED."
2,2022-01-04 13:31:32,Client012,2022-01-04,By Phone,30 Day,UHC,Yes,COTTONWOOD HOSP DISCHARGED 12.31.2021,UNKNOWN,No falls,...,Yes,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,ALL IS WELL
3,2022-01-06 11:56:52,Client022,2022-01-06,By Phone,90 Day,UHC,No,NaN,NaN,No falls,...,Yes,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,NEW FCG IS APPLYING CARMEN HERNANDEZ
4,2022-01-10 11:03:08,Client087,2022-01-10,By Phone,30 Day,UHC,No,NaN,NaN,No falls,...,Yes,NaN,NaN,NaN,No,NaN,Yes,NaN,ALL IS WELL,RAMS AND RAISE WALKER


In [21]:
dfadl.head()

,DeIdentify ID,ProviderID,PayerGroup,Caregiver ID,VisitDuration,ServiceTypeName,VisitDate,ActualTimeIn,ActualTimeOut,txtBreakTimesValue,VisitDate2,ScheduledTimeIn,ScheduledTimeOut,TaskCodeGroupName,HasBeenDone,Reason,Comments,NoSignature,RectangleVoiceCaptured,SignatureDate
0,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Clean Bathroom (30),Y,NaN,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM
1,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Clean Kitchen (31),Y,NaN,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM
2,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Dust (32),Y,NaN,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM
3,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Laundry (36),Y,NaN,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM
4,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Linen Change (37),N,Performed on another day,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM


In [22]:
dfadl['date'] = pd.to_datetime(dfadl['VisitDate'])
dfadl['date'] = dfadl['date'].dt.date
dfadl['month_year'] = dfadl['date'].map(lambda x: 100*x.year + x.month)

In [23]:
tempdate = pd.to_datetime(dfcrf['Time'])
dfcrf['date'] = tempdate.dt.date
dfcrf['month_year'] = dfcrf['date'].map(lambda x: 100*x.year + x.month)

In [25]:
dfcrf['client'] = dfcrf['Masked Client ID']
dfadl['client'] = dfadl['DeIdentify ID']

In [26]:
mergedcrfadl = pd.merge(dfadl, dfcrf, on = ["client","month_year"])
mergedcrfadl
mergedcrfadl = mergedcrfadl.sample(n=40000)

In [27]:
from pmdarima import auto_arima
import warnings
warnings.filterwarnings("ignore")

In [28]:
#mergedcrfadl['Any_Hospitalizations_in_the_last_30_days']=mergedcrfadl['Any_Hospitalizations_in_the_last_30_days'].eq('Yes').mul(1)
#mergedcrfadl.Any_Hospitalizations_in_the_last_30_days=pd.Series(np.where(mergedcrfadl.Any_Hospitalizations_in_the_last_30_days.values == 'Yes', 1, 0),mergedcrfadl.index)
mergedcrfadl['Any Hospitalizations in the last 30 days'].replace(['No', 'Yes'],
                        [0, 1], inplace=True)
mergedcrfadl['Any Hospitalizations in the last 30 days'].unique()


array([0, 1])

In [29]:
mergedcrfadl=mergedcrfadl.drop(['Date of Review', 'DeIdentify ID', 'Masked Client ID', 'Comments'], axis = 1)
mergedcrfadl

,ProviderID,PayerGroup,Caregiver ID,VisitDuration,ServiceTypeName,VisitDate,ActualTimeIn,ActualTimeOut,txtBreakTimesValue,VisitDate2,...,Detail any issues,Are the cargivers able to perform all the tasks on your care plan,Detail any issues.1,Are there any upcoming schedule changes?,Details of Schedule changes,Are you happy with the services being provided?,If No make notes here,Any Questions or Concerns?,Case Manager Notes,date_y
33746,127318.0,United Community Plan / United Community Plan ...,Caregiver088,4 h 30m,AC Family Resident,2022-05-23,16:00:00,20:30:00,NaN,2022-05-23,...,NaN,NaN,NaN,No,NaN,No,NOT TOO WELL,NOT AT THIS TIME,WILL SPEAK WITH FCG RE ASSISTING GRANDPARENTS,2022-05-25
76639,486278.0,DDD / DDD / DDD,Caregiver163,1 h,DDD Habilitation,2022-08-31,09:00:00,10:00:00,NaN,2022-08-31,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NONE,Caregiver and Client are doing great,2022-08-18
68991,127318.0,United Community Plan / United Community Plan ...,Caregiver151,4 h,AC Family Resident,2022-02-11,11:00:00,15:00:00,NaN,2022-02-11,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,ALL IS WELL,2022-02-16
17432,127318.0,Mercy Care / Mercy Care / Mercy Care,Caregiver038,3 h,AC Family Resident,2022-04-20,18:00:00,21:00:00,NaN,2022-04-20,...,NaN,NaN,NaN,No,NaN,Yes,NaN,Was asking about a pay raise. Was offered a hi...,Client is doing well. Sent email to crissy abo...,2022-04-21
34935,127318.0,United Community Plan / United Community Plan ...,Caregiver092,4 h,AC Family non resident,2022-02-28,11:30:00,15:30:00,NaN,2022-02-28,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,ALL IS WELL,2022-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65343,127318.0,United Community Plan / United Community Plan ...,Caregiver145,3 h,AC Family Resident,2022-02-23,05:00:00,08:00:00,NaN,2022-02-23,...,NaN,NaN,NaN,Yes,"HAD TO GO TO CALIFORNIA/ DAD, EXTREMLY SICK PE...",Yes,NaN,NOT AT THIS TIME,HAS A DR. APPT TOMORROE 2.8.2022,2022-02-07
87251,127318.0,Mercy Care / Mercy Care / Mercy Care,Caregiver180,3 h,AC Family Resident,2022-03-16,07:00:00,10:00:00,NaN,2022-03-16,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,ALL IS WELL,2022-03-17
42968,127318.0,BANNER - UFC / BANNER - UFC / BANNER - UFC,Caregiver104,6 h,AC Family Resident,2022-01-23,09:00:00,15:00:00,NaN,2022-01-23,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,ALL IS WELL,2022-01-26
60646,127318.0,United Community Plan / United Community Plan ...,Caregiver136,6 h,AC Attendant Care,2022-02-06,06:00:00,12:00:00,NaN,2022-02-06,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,ALL IS WELL,2022-02-21


In [30]:
mergedcrfadl=mergedcrfadl.drop(['ProviderID', 'PayerGroup', 'VisitDuration','VisitDate','txtBreakTimesValue', 'VisitDate2', 'ScheduledTimeIn','Date and Place of last HbA1C Check',
       'Date and place of last Mammogram (if female)',
       'Date and place of last Colorectal Exam',
       'Date and place of last time you had your cholesterol checked',
       'Date and place of last Bone Density Scan',
       'Does your health plan meet your needs?',
       'Has the Care Plan been reviewed with the Client',
       'Any changes to current care plan', 'Notes on changes to Care Plan',
       'Do your caregivers arrive on time?', 'Details of timekeeping issues',
       'Does your Caregiver carry out the care plan satisfactory',
       'Detail any issues',
       'Are the cargivers able to perform all the tasks on your care plan',
       'Detail any issues.1', 'Are there any upcoming schedule changes?',
       'Details of Schedule changes',
       'Are you happy with the services being provided?',
       'If No make notes here', 'Any Questions or Concerns?',
       'Case Manager Notes', 'date_y', 'Notes on changes to medical conditions','Notes on new medical devises', 'Any changes to Mental Status',
       'Notes on changes to mental status', 'How is your vision?',
       'How is your hearing', 'Do you take your medications',
       'Do you fill all your prescriptions?',
       'How would you describe your diet',
       'How much water do you drink in a day?','Note on Hospitalization (date and reason)',
       'How can we help prevent a return to hospital?','Which Review is this',
       'Insurance',
       'Have you fallen in:', 'Did your fall result in',
       'Matrix (Do you worry about falling?)',
       'Matrix (Do you have a Living Will?)', 'Matrix (Do you have a  DNR?)',
       'Do you have a POA', 'Do you need help with DNR, Living Will or POA',
       'Note on help needed?',
       'Are there any changes to your home environment since the last Intake/Review?',
       'Condition of the Home',
       'Are you receiving Home Health Services', 'If so from which Agency',
       'Are you expecting a Nurse or Therapist to visit you at home?',
       'What will the Nurse/Therapist do?',
       'Any changes to Transportation needs?', 'Notes on transportation needs',
       'Have you had any new DME equipment in the last 90 days?',
       'Notes on new DME equipment', 'Do you need any DME Equipment?',
       'Notes on DME Equipment needed',
       'Any new medical devises in the past 90 days', 'How do you sleep',
       'What do you do about your medical appointments','Where and when did you have your flu shot?','Caregiver ID', 'ServiceTypeName', 'ActualTimeIn', 'ActualTimeOut',
       'ScheduledTimeOut', 'Reason','NoSignature', 'RectangleVoiceCaptured', 'SignatureDate', 'date_x'], axis = 1)
mergedcrfadl.columns

Index(['TaskCodeGroupName', 'HasBeenDone', 'month_year', 'client', 'Time',
       'COVID -19', 'Any Hospitalizations in the last 30 days',
       'Has anyone moved in our out in the last 30 days?',
       'Any new medical conditions in the last 90 days',
       'Have you had your flu shot this year?'],
      dtype='object')

In [31]:
#mergedcrfadl=mergedcrfadl.drop(['COVID -19','Have you had your flu shot this year?','Any new medical conditions in the last 90 days'],axis = 1)
mergedcrfadl_data = pd.get_dummies(mergedcrfadl, columns = ['Has anyone moved in our out in the last 30 days?','client','TaskCodeGroupName', 'HasBeenDone','COVID -19','Have you had your flu shot this year?','Any new medical conditions in the last 90 days'])
#mergedcrfadl_data = mergedcrfadl_data.sample(n=40000)

In [32]:
arima_fit = auto_arima(mergedcrfadl_data['Any Hospitalizations in the last 30 days'], trace = True, suppress_warnings=True)
arima_fit.summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-17272.144, Time=12.81 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-17278.400, Time=7.65 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-17276.952, Time=3.39 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-17276.945, Time=5.28 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-15665.224, Time=1.30 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-17274.894, Time=7.02 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0] intercept
Total fit time: 37.488 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                40000
Model:                        SARIMAX   Log Likelihood                8641.200
Date:                Sat, 10 Dec 2022   AIC                         -17278.400
Time:                        14:26:08   BIC                         -17261.206
Sample:                             0   HQIC                        -17272.957
                              - 40000                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0396    6.6e-06   5998.012      0.000       0.040       0.040
sigma2         0.0380   7.99e-05    475.550      0.000       0.038       0.038
===================================================================================
Ljung-Box (L1) (Q):                   0.55   Jarque-Bera (JB):            836203.28
Prob(Q):                              0.46   Prob(JB):                         0.00
Heteroskedasticity (H):               1.04   Skew:                             4.72
Prob(H) (two-sided):                  0.02   Kurtosis:                        23.31
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number    inf. Standard errors may be unstable.
"""

In [33]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

In [34]:
trainingset = mergedcrfadl_data[:-4000]
testingset = mergedcrfadl_data[-4000:]

In [35]:
arima_model = sm.tsa.arima.ARIMA(trainingset['Any Hospitalizations in the last 30 days'],order=(1,0,0))
arima_model = arima_model.fit()
arima_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                          SARIMAX Results                                           
====================================================================================================
Dep. Variable:     Any Hospitalizations in the last 30 days   No. Observations:                36000
Model:                                       ARIMA(1, 0, 0)   Log Likelihood                7676.997
Date:                                      Sat, 10 Dec 2022   AIC                         -15347.994
Time:                                              14:26:21   BIC                         -15322.520
Sample:                                                   0   HQIC                        -15339.890
                                                    - 36000                                         
Covariance Type:                                        opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0398      0.349      0.114      0.909      -0.643       0.723
ar.L1         -0.0029      0.006     -0.499      0.618      -0.014       0.009
sigma2         0.0382      0.029      1.316      0.188      -0.019       0.095
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):            742811.51
Prob(Q):                              1.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.05   Skew:                             4.71
Prob(H) (two-sided):                  0.01   Kurtosis:                        23.16
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [36]:
start = len(trainingset)
end = len(trainingset)+len(testingset)-1
predctn = arima_model.predict(start=start,end=end,type='levels')
print(predctn)
predctn.unique()

36000    0.039922
36001    0.039805
36002    0.039805
36003    0.039805
36004    0.039805
           ...   
39995    0.039805
39996    0.039805
39997    0.039805
39998    0.039805
39999    0.039805
Name: predicted_mean, Length: 4000, dtype: float64


array([0.03992198, 0.03980468, 0.03980502, 0.03980502, 0.03980502,
       0.03980502, 0.03980502])

In [37]:
print(trainingset['Any Hospitalizations in the last 30 days'].mean())
ptemp = []
for p in predctn:
  if p > trainingset['Any Hospitalizations in the last 30 days'].mean():
    ptemp.append(1)
  else:
    ptemp.append(0)

predctn = ptemp

0.03980555555555555


In [38]:
 from sklearn.metrics import f1_score

In [41]:
truevals = testingset['Any Hospitalizations in the last 30 days']
print(f1_score(truevals, predctn, average='macro'))
#f1 score
from sklearn.metrics import accuracy_score
print(accuracy_score(truevals, predctn))

0.4903809402471652
0.96225
